## Starting with Michael Wood-Vasey's notebook as a starting point
## Craig Lage - 24Aug18

## Make a brighter-fatter kernel from a set of high-intensity flats measured at UC Davis.
Michael Wood-Vasey  (2018-08-21)

1. Set up a basic repo with the raw flats
2. Use  `MakeBrighterFatterKernelTask` to construct a BF kernel
3. Visualize the kernels and compare PhoSim, ImSim, HSC, and lab-based ITL.

With substantial help from Merlin Fisher-Levine.

In [1]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib


lsst_distrib          16.0+1     	current o_latest v16_0 setup


In [2]:
import numpy as np
import matplotlib.pyplot as plt
# if running stack v16.0, silence a long matplotlib Agg warning with:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
# Test to make sure that we can import obs_lsstCam
import lsst.obs.base
import lsst.obs.lsstCam

In [4]:
from makeBrighterFatterKernel import MakeBrighterFatterKernelTask
# Just copied the .py file into my directory.  Is this OK?
from lsst.daf.persistence import Butler
from lsst.pipe.tasks.ingest import IngestTask


In [5]:
!pwd

/sandbox/cslage/Research/LSST/code/notebooks/DC2_Notebooks


## Try to Ingest some flats from UC Davis measurements

In [6]:
!ls /mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_100_20180514??????.fits

/mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_100_20180514141655.fits


In [7]:
!rm -rf ucd_repo
! mkdir ucd_repo
! echo "lsst.obs.lsstCam.LsstCamMapper" > ucd_repo/_mapper

# Ingest the  flats. The ?00 and ?01 are the flat pairs
! ingestImages.py ucd_repo /mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_?00_20180514??????.fits --mode link
! ingestImages.py ucd_repo /mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_?01_20180514??????.fits --mode link

root INFO: Loading config overrride file '/sandbox/lsstshared/v16_0/obs_lsstCam/config/ingest.py'
LsstCamMapper WARN: Unable to find calib root directory
CameraMapper INFO: Loading Posix exposure registry from /sandbox/cslage/Research/LSST/code/notebooks/DC2_Notebooks/ucd_repo
ingest INFO: /mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_100_20180514141655.fits --<link>--> /sandbox/cslage/Research/LSST/code/notebooks/DC2_Notebooks/ucd_repo/raw/100/R21/00000100-R21-S11-det085-000.fits
ingest INFO: /mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_200_20180514143548.fits --<link>--> /sandbox/cslage/Research/LSST/code/notebooks/DC2_Notebooks/ucd_repo/raw/200/R21/00000200-R21-S11-det085-000.fits
ingest INFO: /mnt/storm-lsst/GUI/20180514_002_flats_3/ITL-3800C-002_flat_flat_300_20180514145807.fits --<link>--> /sandbox/cslage/Research/LSST/code/notebooks/DC2_Notebooks/ucd_repo/raw/300/R21/00000300-R21-S11-det085-000.fits
ingest INFO: /mnt/storm-lsst/GUI/20180514

In [9]:
butler = Butler('ucd_repo')

So I got this far, and it seems to have successfully ingested the images.  So far, all I have done is:

     (1) edit obs_lsstCam/config/ingest.py to use some different header values
     (2) edit obs_lsstCam/python/lsst/obs/lsstCam/ingest.py to work with these different values and 
     to fudge the raft and sensor IDs.
    

It's encouraging that I created a butler, but is the right information there???

In [10]:
print(butler.queryMetadata('src', ['visit', 'detector', 'filter']))
print(butler.datasetExists('src'))


[(100, 85, 'R'), (101, 85, 'R'), (200, 85, 'R'), (201, 85, 'R'), (300, 85, 'R'), (301, 85, 'R'), (400, 85, 'R'), (401, 85, 'R'), (500, 85, 'R'), (501, 85, 'R')]
False


Why doesn't the dataset exist???

Try calculating the brighter-fatter kernel using `MakeBrighterFatterKernelTask`. 

In [11]:
args = ['ucd_repo', '--rerun', 'test',
        '--id', 'detector={}',
        '--visit-pairs', '100,101', '200,201', '300,301', '400,401', '500,501',
        '-c',
        'xcorrCheckRejectLevel=2', 'doCalcGains=True',
        '--clobber-config', '--clobber-versions'
       ]

command_line = 'makeBrighterFatterKernel.py ' + ' '.join(args)
print(command_line)

ucd_pb_struct = MakeBrighterFatterKernelTask.parseAndRun(args=args)

makeBrighterFatterKernel.py ucd_repo --rerun test --id detector={} --visit-pairs 100,101 200,201 300,301 400,401 500,501 -c xcorrCheckRejectLevel=2 doCalcGains=True --clobber-config --clobber-versions


usage: ipykernel_launcher.py input [options]
ipykernel_launcher.py: error: 'Cannot get keys for datasetType brighterFatterKernelNew at level None'


SystemExit: 2

Ignore warnings.